# OAO Reader - "OnAndOn" format of the Motion GPS

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://logiqx.github.io/gps-wizard/) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import os
import sys
import time

import numpy as np

import unittest

from base_reader import BaseReader

## Main Class

In [2]:
class OaoReader(BaseReader):
    '''OAO file - "OnAndOn" format of the Motion GPS'''

    def __init__(self, filename):
        '''Basic init just records the filename'''

        super().__init__(filename)


    def load(self, ignoreChecksums=False):
        '''Load file into memory'''

        with open(self.filename, "rb") as f:
            self.buffer = f.read()
            
            # The memoryview will be used to access the buffer without intermediate copying
            self.bufferView = memoryview(self.buffer)

            if len(self.buffer) > 0:
                self.readData(ignoreChecksums=ignoreChecksums)


    def readData(self, ignoreChecksums=False):
        '''Read GNSS frames into memory'''

        fileSize = len(self.buffer)

        # Lean buffer will only contain the geodetic navigation data, minus start / end tokens and checksums
        leanBuffer = bytearray()

        numRecords = 0

        while self.bufferPtr < fileSize:
            mode, payload = self.readFrame(ignoreChecksums=ignoreChecksums)
            
            if mode in (0x0ad4, 0x0ad5):
                # Use += because it is faster than extend - see https://stackoverflow.com/q/40004517
                leanBuffer += payload
                numRecords += 1
                       
        if self.bufferPtr != fileSize:
            raise ValueError('File appears to be truncated')

        dtype = self.getDatatype()
        
        self.rawData = np.frombuffer(leanBuffer, dtype=dtype, count=numRecords)
        
        self.consumeRawData()

        self.summarise()


    def readFrame(self, ignoreChecksums=False):
        '''Read OAO frame from buffer'''

        offset = self.bufferPtr

        mode = self.readUnsigned16LE()
        expectedChecksum = self.readUnsigned16LE()

        if mode == 0x0ad0:
            length = 512
        elif mode == 0x0ad1:
            length = 12
        elif mode == 0x0ad2 or mode == 0x0ad3:
            length = 34
        elif mode == 0x0ad4 or mode == 0x0ad5:
            length = 52
        else:
            raise ValueError('Unexpected mode 0x{:x} - offset 0x{:08x}'.format(mode, offset))

        # Main frame content
        payload = self.readBytes(length - 4)

        # Evaluate checksum
        if ignoreChecksums == False:
            actualChecksum = self.calculateChecksum(mode, payload)
            if actualChecksum != expectedChecksum:
                raise ValueError('Checksum difference - {} vs {}'.format(actualChecksum, expectedChecksum))

        return mode, payload


    def calculateChecksum(self, mode, payload):
        '''Calculate Fletcher's checksum for the OAO frame'''

        checksum_a = checksum_b = 0
        
        # Sum the two first bytes
        for i in range(2):
            checksum_a += mode
            checksum_b += checksum_a
            mode >>= 8

        # Sum the payload bytes
        for byte in payload:
            checksum_a += byte
            checksum_b += checksum_a

        # Bit masking has been deferred until the end
        return (checksum_b & 0xff) << 8 | checksum_a & 0xff


    def getDatatype(self):
        '''Get the datatype of the GNSS frames'''

        # Standard GNSS frame (mode = 0x0ad4 or 0x0ad5) - 48 bytes excluding the mode and checksum
        dtype = [
            ('latitude', '<i4'),
            ('longitude', '<i4'),
            ('altitude_msl', '<i4'),
            ('speed', '<u4'),
            ('course', '<u4'),
            ('utc_time', '<u8'),
            ('fix', 'u1'),
            ('satellites', 'u1'),
            ('speed_accuracy', '<u4'),
            ('horizontal_accuracy', '<u4'),
            ('vertical_accuracy', '<u4'),
            ('heading_accuracy', '<u4'),
            ('hdop', '<u2')
        ]

        return np.dtype(dtype)


    def consumeRawData(self):
        '''Consume data - convert to standard data structure'''

        # Create new track object
        track = self.addTrack()

        # Convert latitude and longitude to decimal
        track.data['lat'] = np.divide(self.rawData['latitude'], 10000000, dtype='float64')
        track.data['lon'] = np.divide(self.rawData['longitude'], 10000000, dtype='float64')

        # Convert elevation from cm to m
        track.data['ele'] = np.divide(self.rawData['altitude_msl'], 1000, dtype='float64')

        # Convert SOG from mm/s to m/s
        track.data['sog'] = np.divide(self.rawData['speed'], 1000, dtype='float32')

        # Convert COG from integer to decimal
        track.data['cog'] = np.round(np.divide(self.rawData['course'], 100000, dtype='float32'), 3)

        # Convert timestamp from ms to s
        track.data['ts'] = np.divide(self.rawData['utc_time'], 1000, dtype='float64')

        # Copy fix and satellites
        track.data['fix'] = self.rawData['fix']
        track.data['sat'] = self.rawData['satellites']

        # Convert sAcc from mm/s to m/s
        track.data['sacc'] = np.divide(self.rawData['speed_accuracy'], 1000, dtype='float32')

        # Convert hAcc and vAcc from mm to m
        track.data['hacc'] = np.divide(self.rawData['horizontal_accuracy'], 1000, dtype='float32')
        track.data['vacc'] = np.divide(self.rawData['vertical_accuracy'], 1000, dtype='float32')

        # Convert cAcc to degrees
        track.data['cacc'] = np.round(np.divide(self.rawData['heading_accuracy'], 100000, dtype='float32'), 3)

        # Convert HDOP from integer to decimal
        track.data['hdop'] = np.divide(self.rawData['hdop'], 100, dtype='float32')

## Unit Tests

In [3]:
class TestMiniData(unittest.TestCase):
    '''Class to test Motion Mini data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        for fieldName in miniReader.tracks[0].data:
            self.assertEqual(miniReader.tracks[0].data[fieldName].size, 47980)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        self.assertEqual(miniReader.tracks[0].data['hdop'].min(), np.float32(0.62))
        self.assertEqual(miniReader.tracks[0].data['hdop'].max(), np.float32(99.99))


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(miniReader.tracks[0].data['sat'].min(), np.uint8(0))
        self.assertEqual(miniReader.tracks[0].data['sat'].max(), np.uint8(18))


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(miniReader.tracks[0].data['ts'].min(), np.float64(1649672228.0))
        self.assertEqual(miniReader.tracks[0].data['ts'].max(), np.float64(1649677025.9))


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(miniReader.tracks[0].data['lat'].min(), np.float64(50.5710305))
        self.assertEqual(miniReader.tracks[0].data['lat'].max(), np.float64(50.5833157))


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(miniReader.tracks[0].data['lon'].min(), np.float64(-2.462027))
        self.assertEqual(miniReader.tracks[0].data['lon'].max(), np.float64(-2.4563075))


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(miniReader.tracks[0].data['ele'].min(), np.float64(-3.169))
        self.assertEqual(miniReader.tracks[0].data['ele'].max(), np.float64(13.548))


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(miniReader.tracks[0].data['sog'].min(), np.float32(0.000))
        self.assertEqual(miniReader.tracks[0].data['sog'].max(), np.float32(17.142))


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(miniReader.tracks[0].data['cog'].min(), np.float32(0.001))
        self.assertEqual(miniReader.tracks[0].data['cog'].max(), np.float32(359.996))


    def testFix(self):
        '''Test the fix is as expected'''

        self.assertEqual(miniReader.tracks[0].data['fix'].min(), np.uint8(0))
        self.assertEqual(miniReader.tracks[0].data['fix'].max(), np.uint8(3))


    def testSacc(self):
        '''Test the speed accuracy is as expected'''

        self.assertEqual(miniReader.tracks[0].data['sacc'].min(), np.float32(0.092))
        self.assertEqual(miniReader.tracks[0].data['sacc'].max(), np.float32(2.118))


    def testHacc(self):
        '''Test the horizontal accuracy is as expected'''

        self.assertEqual(miniReader.tracks[0].data['hacc'].min(), np.float32(0.395))
        self.assertEqual(miniReader.tracks[0].data['hacc'].max(), np.float32(12.051))


    def testVacc(self):
        '''Test the vertical accuracy is as expected'''

        self.assertEqual(miniReader.tracks[0].data['vacc'].min(), np.float32(0.524))
        self.assertEqual(miniReader.tracks[0].data['vacc'].max(), np.float32(11.574))


    def testCacc(self):
        '''Test the course accuracy is as expected'''

        self.assertEqual(miniReader.tracks[0].data['cacc'].min(), np.float32(0.401))
        self.assertEqual(miniReader.tracks[0].data['cacc'].max(), np.float32(45.016))

In [4]:
class TestEspData(unittest.TestCase):
    '''Class to test ESP-GPS data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        for fieldName in espReader.tracks[0].data:
            self.assertEqual(espReader.tracks[0].data[fieldName].size, 43444)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        # Note: HDOP in this file contains NAV-PVT.pDOP instead of NAV-DOP.hDOP
        self.assertEqual(espReader.tracks[0].data['hdop'].min(), np.float32(0.96))
        self.assertEqual(espReader.tracks[0].data['hdop'].max(), np.float32(1.88))


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(espReader.tracks[0].data['sat'].min(), np.uint8(9))
        self.assertEqual(espReader.tracks[0].data['sat'].max(), np.uint8(18))


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(espReader.tracks[0].data['ts'].min(), np.float64(1640864825.3))
        self.assertEqual(espReader.tracks[0].data['ts'].max(), np.float64(1640869169.6))


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(espReader.tracks[0].data['lat'].min(), np.float64(51.6993591))
        self.assertEqual(espReader.tracks[0].data['lat'].max(), np.float64(51.7087559))


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(espReader.tracks[0].data['lon'].min(), np.float64(4.0860906))
        self.assertEqual(espReader.tracks[0].data['lon'].max(), np.float64(4.0890658))


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(espReader.tracks[0].data['ele'].min(), np.float64(-3.159))
        self.assertEqual(espReader.tracks[0].data['ele'].max(), np.float64(11.978))


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(espReader.tracks[0].data['sog'].min(), np.float32(0.000))
        self.assertEqual(espReader.tracks[0].data['sog'].max(), np.float32(18.169))


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(espReader.tracks[0].data['cog'].min(), np.float32(0.011))
        self.assertEqual(espReader.tracks[0].data['cog'].max(), np.float32(359.995))


    def testFix(self):
        '''Test the fix is as expected'''

        self.assertEqual(espReader.tracks[0].data['fix'].min(), np.uint8(3))
        self.assertEqual(espReader.tracks[0].data['fix'].max(), np.uint8(3))


    def testSacc(self):
        '''Test the speed accuracy is as expected'''

        self.assertEqual(espReader.tracks[0].data['sacc'].min(), np.float32(0.051))
        self.assertEqual(espReader.tracks[0].data['sacc'].max(), np.float32(0.771))


    def testHacc(self):
        '''Test the horizontal accuracy is as expected'''

        self.assertEqual(espReader.tracks[0].data['hacc'].min(), np.float32(0.169))
        self.assertEqual(espReader.tracks[0].data['hacc'].max(), np.float32(2.508))


    def testVacc(self):
        '''Test the vertical accuracy is as expected'''

        self.assertEqual(espReader.tracks[0].data['vacc'].min(), np.float32(0.207))
        self.assertEqual(espReader.tracks[0].data['vacc'].max(), np.float32(2.582))


    def testCacc(self):
        '''Test the course accuracy is as expected'''

        self.assertEqual(espReader.tracks[0].data['cacc'].min(), np.float32(0.239))
        self.assertEqual(espReader.tracks[0].data['cacc'].max(), np.float32(44.999))

In [5]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    filename = os.path.join(projdir, 'sessions', '20220411', 'Motion_Mini_10Hz_0470_2022-04-11-1117.oao')
    miniReader = OaoReader(filename)

    filename = os.path.join(projdir, 'sessions', '20211230-esp', 'Head_L_7C9EBDFAF5C8_007.oao')
    espReader = OaoReader(filename)

    pc1 = time.perf_counter()
    miniReader.load()
    espReader.load()
    pc2 = time.perf_counter()

    print("\nOAO files loaded in in %0.2f seconds" % (pc2 - pc1))


OAO files loaded in in 0.69 seconds


In [6]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

............................
----------------------------------------------------------------------
Ran 28 tests in 0.018s

OK
